# Bệnh dịch Tân sinh viên

*Mô hình hóa và mô phỏng bằng Python*

Bản quyền 2021 Allen Downey

Giấy phép: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [16]:
# cài đặt Pint nếu cần

try:
    import pint
except ImportError:
    !pip install pint

In [17]:
# tải về modsim.py nếu cần

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://raw.githubusercontent.com/AllenDowney/' +
         'ModSimPy/master/modsim.py')

In [18]:
# import functions from modsim

from modsim import *

In [24]:
download('https://github.com/AllenDowney/ModSimPy/raw/master/' +
         'chap11.py')

In [25]:
# nhập mã lệnh từ các sổ notebooks trước

from chap11 import make_system
from chap11 import update_func
from chap11 import run_simulation

[Nhấn vào đây để chạy nghiên cứu cụ thể này trên Colab](https://colab.research.google.com/github/AllenDowney/ModSimPy/blob/master/examples/plague.ipynb)

Nghiên cứu cụ thể này tiếp tục từ điểm dừng ở Chương 12.

In [26]:
def add_immunization(system, fraction):
    system.init.S -= fraction
    system.init.R += fraction

In [27]:
tc = 3             # thời gian giữa các lượt tiếp xúc, đơn vị ngày 
tr = 4             # thời gian hồi phục, đơn vị ngày

beta = 1 / tc      # tốc độ tiếp xúc, đơn vị 1/ngày
gamma = 1 / tr     # tốc độ hồi phục, đơn vị 1/ngày

system = make_system(beta, gamma)

In [28]:
def calc_total_infected(results, system):
    s_0 = results.S[0]
    s_end = results.S[system.t_end]
    return s_0 - s_end

## Rửa tay

Giả sử bạn là Trưởng ban Đời sống sinh viên, và bạn chỉ có quỹ \$1200  để chống lại Bệnh dịch tân sinh viên. Bạn có hai phương án tiêu số tiền này:

1.  Bạn có thể chi trả việc tiêm vắc-xin với giá \$100 mỗi liều.

2.  Bạn có thể tiêu tiền để vận động tuyên truyền cho sinh viên rửa tay
    thường xuyên.

Ta đã thấy rằng có thể mô hình hóa hiệu ứng của vắc-xin như thế nào rồi. 
Giờ ta hãy nghĩ về vận động tuyên truyền rửa tay. Ta sẽ phải trả lời 
hai câu hỏi:

1.  Làm thế nào để đưa hiệu ứng của việc rửa tay vào mô hình này?

2.  Làm thế nào để định lượng hiệu ứng của số tiền dành cho tổ chức vận động
    tuyên truyền cho rửa tay?

Để đơn giản, hãy giả sử rằng ta đã có dữ liệu của một đợt vận động tương tự
ở một trường học khác, trong đó cho thấy rằng một đợt vận động được đầu tư đầy đủ
sẽ làm thay đổi hành động của cộng đồng sinh viên đến mức giảm được tỉ lệ nhiễm bớt được 20%.

Xét về khía cạnh mô hình, việc rửa tay có hiệu ứng làm giảm `beta`.
Đó không phải cách duy nhất mà ta có thẻ đưa hiệu ứng vào, nhưng nó có vẻ
hợp lý và dễ thực hiện.

Bây giờ ta phải mô hình hóa mối liên hệ giữa số tiền chi và
hiệu quả của cuộc vận động. Một lần nữa, hãy giả sử rằng dữ liệu
từ ngôi trường khác cho thấy:

-   Nếu ta chi \$500 cho các poster, vật liệu và thời gian lao động, thì ta có thể
    làm thay đổi biểu hiện của học sinh theo cách làm giảm giá trị hiệu quả của `beta` đi 10%.

-   Nếu ta chi \$1000, mức tăng tổng cộng của `beta` sẽ gần bằng 20%.

-   Quá \$1000, những chi tiêu vượt thêm chỉ mang lại ít lợi ích.

## Hàm logit

Để mô hình hóa hiệu ứng của cuộc vận động rửa tay, tôi sẽ dùng một [hàm logit tổng quát](https://en.wikipedia.org/wiki/Generalised_logistic_function) (GLF), đây là một hàm tiện dụng cho mô hình hóa các đường cong có hình dạng chung như chữ S. Các tham số của hàm GLF tương ứng với các đặc điểm của đường cong theo cách cho phép ta dễ dàng tìm được một hàm để có hình dáng như mong muốn, dựa trên số liệu hay thông tin nền về kịch bản.

In [29]:
from numpy import exp

def logistic(x, A=0, B=1, C=1, M=0, K=1, Q=1, nu=1):
    """Tính hàm logit tổng quát.
    
    A: kiểm soát cận dưới
    B: kiểm soát độ dốc chuyển giao 
    C: theo tôi thì tham số này vô dụng
    M: kiểm soát vị trí của chuyển giao
    K: kiểm soát cận dưới
    Q: dịch chuyển giao qua trái hoặc phải
    nu: ảnh hưởng tính đối xứng của chuyển giao
    
    trả lại: float hoặc array
    """
    exponent = -B * (x - M)
    denom = C + Q * exp(exponent)
    return A + (K-A) / denom ** (1/nu)

Mảng sau đây thể hiện khoảng các mức chi khả dĩ.

In [30]:
spending = linspace(0, 1200, 21)

`compute_factor` tính mức giảm `beta` với một khoản chi vận động cho trước.

`M` được chọn sao cho chuyển giao nằm ở quanh mức \$500.

`K` là mức giảm tối đa của `beta`, 20%.

`B` được chọn qua cách thử sai nhằm thu được một đường cong khả dĩ.

In [31]:
def compute_factor(spending):
    """Hệ số chiết giảm như một hàm phụ thuộc vào mức chi.
    
    spending: mức chi tính theo đôla từ 0 đến 1200
    
    trả lại: tỉ lệ phần giảm của beta
    """
    return logistic(spending, M=500, K=0.2, B=0.01)

Sau đây là hình dáng của hàm lập được.

In [32]:
percent_reduction = compute_factor(spending) * 100

make_series(spending, percent_reduction).plot()

decorate(xlabel='Mức chi vận động tuyên truyền rửa tay (USD)',
         ylabel='Phần trăm mức độ giảm tỉ lệ nhiễm',
         title='Hiệu ứng của việc rửa tay đối với tỉ lệ nhiễm')

Kết quả là hàm số sau, hàm này
nhận vào tham số là mức chi và trả lại `factor`, là tỉ lệ
mà `beta` bị giảm:

In [33]:
def compute_factor(spending):
    return logistic(spending, M=500, K=0.2, B=0.01)

Tôi dùng `compute_factor` để viết `add_hand_washing`, hàm này nhận vào
mối đối tượng `System` và một khoản tiền, rồi thay đổi `system.beta` để mô hình hóa
hiệu ứng của phong trào rửa tay:

In [34]:
def add_hand_washing(system, spending):
    factor = compute_factor(spending)
    system.beta *= (1 - factor)

Bây giờ ta có thể quét một khoảng các giá trị `spending` và dùng mô phỏng
để tính được hiệu ứng:

In [35]:
def sweep_hand_washing(spending_array):
    sweep = SweepSeries()
    
    for spending in spending_array:
        system = make_system(beta, gamma)
        add_hand_washing(system, spending)
        results = run_simulation(system, update_func)
        sweep[spending] = calc_total_infected(results, system)
        
    return sweep

Đây là cách ta chạy hàm này:

In [36]:
from numpy import linspace

spending_array = linspace(0, 1200, 20)
infected_sweep2 = sweep_hand_washing(spending_array)

Hình vẽ dưới đây cho thấy kết quả. 

In [37]:
infected_sweep2.plot()

decorate(xlabel='Mức chi vận động tuyên truyền rửa tay (USD)',
         ylabel='Tỉ lệ nhiễm tổng cộng',
         title='Hiệu ứng của việc rửa tay đối với tỉ lệ nhiễm')

Dưới mức \$200, cuộc vận động ít hiệu quả. 

Ở mức \$800 nó có hiệu quả đáng kể, giảm tỉ lệ nhiễm tổng cộng từ hơn 45% xuống còn khoảng 20%. 

Trên mức \$800, những lợi ích phụ thêm là rất nhỏ.

## Tối ưu hóa

Ta hãy tổng hợp tất cả lại. Với một quỹ cố định \$1200, ta phải quyết định
xem cần mua bao nhiêu bao nhiêu liều vắc-xin và phải tiêu bao nhiêu cho
vận động tuyên truyền rửa tay.

Sau đây là các thông số:

In [38]:
num_students = 90
budget = 1200
price_per_dose = 100
max_doses = int(budget / price_per_dose)
max_doses

Tỉ số `budget/price_per_dose` có thể không phải là một số nguyên. `int`
là hàm dựng sẵn để quy đổi các số thành số nguyên, làm tròn xuống.

Ta sẽ quét một loạt các liều lượng khả dĩ:

In [39]:
dose_array = linrange(max_doses)

Ở ví dụ này ta gọi `linrange` với chỉ một đối số; nó trả lại một mảng NumPy array với các số nguyên từ 0 đến `max_doses`, bao gồm cả hai đầu.

Sau đó ta chạy mô phỏng cho từng phần tử của `dose_array`:

In [40]:
def sweep_doses(dose_array):
    sweep = SweepSeries()
    
    for doses in dose_array:
        fraction = doses / num_students
        spending = budget - doses * price_per_dose
        
        system = make_system(beta, gamma)
        add_immunization(system, fraction)
        add_hand_washing(system, spending)
        
        results = run_simulation(system, update_func)
        sweep[doses] = calc_total_infected(results, system)

    return sweep

Với từng số liều, ta tính tỉ lệ sinh viên mà ta có thể
gây miễn dịch, `fraction` và số tiền còn lại có thể tiêu cho cuộc vận động,
`spending`. Sau đó ta chạy mô phỏng với những đại lượng này
và lưu lại số ca nhiễm.

Hình dưới đây cho thấy kết quả.

In [41]:
infected_sweep3 = sweep_doses(dose_array)

In [42]:
infected_sweep3.plot()

decorate(xlabel='Số liều vắc-xin',
         ylabel='Tổng tỉ lệ nhiễm',
         title='Tổng nhiễm so với số liều')

Nếu ta không mua liều vắc-xin nào cả và tiêu toàn bộ tiền cho cuộc vận động, thì tỉ lệ nhiễm vào khoảng 19%. Với 4 liều, ta còn khoảng \$800 cho cuộc vận động, và đây là điểm tối ưu giúp cho cực tiểu hóa số sinh viên bị bệnh.

Khi tăng số liều thuốc, ta phải cắt giảm chi tiêu cho vận động,
điều đó hóa ra lại tồi tệ hơn. Nhưng dáng lưu tâm là khi ta áp dụng
trên 10 liều thì ảnh hưởng miễn dịch cộng đồng bắt đầu phát huy tác dụng và
số sinh viên bị bệnh lại giảm xuống.

**Bài tập:** Giả sử giá của vắc-xin giảm xuống chỉ còn $50 mỗi liều. Điều này sẽ ảnh hưởng đến việc mức chi tối ưu cần huy động là bao nhiêu?